In [2]:
import matplotlib.pyplot as plt
from ase.optimize.sciopt import *               
from ase.utils.geometry import *
from ase.lattice.spacegroup import crystal
from ase.visualize import *
from ase.lattice.surface import surface
from ase import Atoms
from ase import io 
from ase.io.cif import read_cif
from ase.io.vasp import write_vasp
from abtem.visualize import show_atoms
from ase.visualize.plot import plot_atoms
from ase.build import add_adsorbate
from ase.io.lammpsdata import write_lammps_data
from ase import neighborlist
from ase.build import molecule
from scipy import sparse
from ase.io.proteindatabank import write_proteindatabank
import subprocess

/home/ahlawat/.local/lib/python3.9/site-packages/ase/utils/geometry.py:8: UserWarning: Moved to ase.geometry and ase.build
  warnings.warn('Moved to ase.geometry and ase.build')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/spacegroup.py:5: UserWarning: Moved to ase.spacegroup
  warnings.warn('Moved to ase.spacegroup')
/home/ahlawat/.local/lib/python3.9/site-packages/ase/lattice/surface.py:17: UserWarning: Moved to ase.build
  warnings.warn('Moved to ase.build')


In [ ]:
! rm *.vasp

# FAPbI3 structures are taken from: http://dx.doi.org/10.1021/ic401215x

### below FA are placed to rectify unit cells

## hexagonal polymorph: https://dx.doi.org/10.5517/cc11hdjg
- Experimental data
    - Formula 	(I3 Pb -)n,n(C H5 N2 +)
    - Crystal details
    - Space group 	P 63 m c (186)
    - Unit cell 	a 8.6603(14)Å b 8.6603(14)Å c 7.9022(6)Å
    - α 90.00° β 90.00° γ 120.00°
    - Cell volume 	513.27
    - Reduced cell 	a 7.902Å b 8.660Å c 8.660Å
    - α 120.000° β 90.000° γ 90.000°
    - Polymorph 	delta polymorph
    - Colour 	yellow

In [41]:
structure = io.read('hexagonal_phase.cif')
structure = sort(structure)
tmp_molecule=[]
j = 0
num_atoms = len(structure.get_chemical_symbols())
i = 0
del_index = []
N_index = []
flag = False

while i < num_atoms:
    if(structure.get_chemical_symbols()[i] == 'C'):
        del_index.append(i)
        molecule = io.read('FA.pdb')
        molecule.set_cell(structure.cell)
        xmin = molecule.get_center_of_mass()[0]
        xmax = structure.positions[i, 0]
        ymin = molecule.get_center_of_mass()[1]
        ymax = structure.positions[i, 1]
        zmin = molecule.get_center_of_mass()[2]
        zmax = structure.positions[i, 2]
        molecule.positions += (xmax - xmin, ymax - ymin, zmax - zmin)   
        if j==0:
            tmp_molecule = molecule 
        else:
            tmp_molecule += molecule
        j = j+1
    i = i + 1
    
k = 0
while k < num_atoms:
    if(structure.get_chemical_symbols()[k] == 'N'):
        del_index.append(k)
    k = k + 1

del structure[del_index]  


FA_replaced_structure = structure + tmp_molecule
FA_replaced_structure = sort(FA_replaced_structure)

view (FA_replaced_structure)


<Popen: returncode: None args: ['/home/ahlawat/miniconda3/bin/python', '-m',...>

### prepare lammps input file and run

In [42]:
unit_cell = FA_replaced_structure
rep1 = 3
rep2 = 3
rep3 = 3
i = 0
j = 0
k = 0
x = 0 
y = 0
supercell = unit_cell.repeat((rep1,rep2,rep3))
supercell = sort(supercell)
num = len(supercell.get_chemical_symbols())
while k < num:
    cutOff = neighborlist.natural_cutoffs(supercell)
    nl = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True)
    nl.update(supercell)
    if(supercell.get_chemical_symbols()[k] == 'H'):
        x = nl.get_neighbors(k)
        y = x[0]
        for j in y:
            if(supercell.get_chemical_symbols()[j] == 'C'):
                supercell.symbols[k] = 'He'          
    k = k + 1
    
supercell = sort(supercell)
view(supercell)
write_proteindatabank('slab.pdb',supercell, write_arrays=True)

# vmd tcl script for generating lammps data file

In [43]:
with open("solute.tcl","w") as f:
    print("""
#!/usr/bin/tclsh
if {[catch {package require topotools 1.1} ver]} {
  vmdcon -error "$ver. This script requires at least TopoTools v1.1. Exiting..."
    quit
}
if {[catch {package require pbctools 2.3} ver]} {
  vmdcon -error "$ver. This script requires at least pbctools v2.3. Exiting..."
    quit
}
set fname slab.pdb
# check for presence of coordinate file
if {! [file exists $fname]} {
  vmdcon -error "Required file '$fname' not available. Exiting..."
    quit
}

mol new $fname autobonds no waitfor all

proc distance {new_pos x y z} {
  set x1 [lindex $new_pos 0]
    set bc [pbc get]
    set lx [lindex $bc 0 0]
    set ly [lindex $bc 0 1]
    set lz [lindex $bc 0 2]
    set y1 [lindex $new_pos 1]
    set z1 [lindex $new_pos 2]
    set dx [expr $x - $x1]
    set dy [expr $y - $y1]
    set dz [expr $z - $z1]
    if {$dx >   $lx * 0.5} { set dx [expr $dx - $lx] }
  if {$dx <= -$lx * 0.5} { set dx [expr $dx + $lx] }
  if {$dy >   $ly * 0.5} { set dy [expr $dy - $ly] }
  if {$dy <= -$ly * 0.5} { set dy [expr $dy + $ly] }
  if {$dz >   $lz * 0.5} { set dz [expr $dz - $lz] }
  if {$dz <= -$lz * 0.5} { set dz [expr $dz + $lz] }
  set dr [expr ($dx)**2+($dy)**2+($dz)**2]
    return $dr
}
############### SOLUTE ##################
set selH [atomselect top {name H}]
$selH set type H
$selH set mass 1.00800
$selH set charge 0.4648
set numH [$selH num]
set posH [$selH get {x y z}]
set indexH [$selH get index]

set selHe [atomselect top {name He}]
$selHe set type He
$selHe set mass 1.00800
$selHe set charge 0.22250000 
set numHe [$selHe num]
set posHe [$selHe get {x y z}]
set indexHe [$selHe get index]

set selC [atomselect top {name C}]
$selC set type C
$selC set mass 12.0100
$selC set charge 0.56710000 
set numC [$selC num]
set posC [$selC get {x y z}]
set indexC [$selC get index]

set selN [atomselect top {name N}]
$selN set type N
$selN set mass 14.0100
$selN set charge -0.82450000 
set numN [$selN num]
set posN [$selN get {x y z}]
set indexN [$selN get index]

set selPb [atomselect top {name Pb}]
$selPb set type Pb
$selPb set mass 207.2
$selPb set charge 2.00
set numPb [$selPb num]
set posPb [$selPb get {x y z}]
set indexPb [$selPb get index]

set selI [atomselect top {name I}]
$selI set type I
$selI set mass 126.90
$selI set charge -1.00
set numI [$selI num]
set posI [$selI get {x y z}]
set indexI [$selI get index]
#######################################
set sel [atomselect top all]
set natoms [molinfo top get numatoms]
set bondDistanceHN 1.2
set bondDistanceNC 2.5
set bondDistanceCHa 1.5
set selText all
set selTemp [atomselect top $selText]
set pos [$selTemp get {x y z}]
set index [$selTemp get index]
$selTemp delete
set bondDistance2HN [expr $bondDistanceHN*$bondDistanceHN]
set bondDistance2NC [expr $bondDistanceNC*$bondDistanceNC]
set bondDistance2CHa [expr $bondDistanceCHa*$bondDistanceCHa]
set blist {}

############ SOLUTE ##################
foreach r $posC ind1 $indexC {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posHe ind2 $indexHe {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2CHa)} { lappend blist [list $ind1 $ind2]}
  }
}
foreach r $posN ind1 $indexN {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posH ind2 $indexH {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2HN)} { lappend blist [list $ind1 $ind2]}
  }
}
foreach r $posN ind1 $indexN {
# Select neighboring atoms.
  foreach {x y z} $r {break}
  foreach new_pos $posC ind2 $indexC {
    set dr [ distance $new_pos $x $y $z ]
      if {($dr < $bondDistance2NC)} { lappend blist [list $ind1 $ind2]}
  }
}

return $blist
set reD {}
foreach b $blist {
  set bPerm [list [lindex $b 1] [lindex $b 0]]
    set match [lsearch $reD $bPerm]
    if {$match == -1} {lappend reD $b}
}
return $reD
topo setbondlist type $reD
topo retypebonds 
vmdcon -info "assigned [topo numbondtypes] bond types to [topo numbonds] bonds:"
vmdcon -info "bondtypes: [topo bondtypenames]"
topo guessangles  
vmdcon -info "assigned [topo numangletypes] angle types to [topo numangles] angles:"
vmdcon -info "angletypes: [topo angletypenames]"
topo guessdihedrals
vmdcon -info "assigned [topo numdihedraltypes] dihedral types to [topo numdihedrals] dihedrals:"
vmdcon -info "dihedraltypes: [topo dihedraltypenames]"
topo guessimpropers
vmdcon -info "assigned [topo numimpropertypes] improper types to [topo numimpropers] impropers:"
vmdcon -info "impropertypes: [topo impropertypenames]"
mol reanalyze top
pbc get
# we use a high-level tool from to multiply the system.
#TopoTools::replicatemol top 4 4 4

# and write out the result as a lammps data file.
topo writelammpsdata data.FAPI full
#topo writegmxtop topol.top

# for easier testing and visualization, we
# also write out copies in .pdb and .psf format.
#animate write pdb solvated.pdb
#animate write psf solvated.psf
# final sanity check the whole system has to be neutral.
$sel delete
set sel [atomselect top all]
set totq [vecsum [join [$sel get charge] { }]]
if {[expr {abs($totq)}] > 0.0005} {
  vmdcon -warning "Total system not neutral: $totq"
}

# done. now exit vmd
quit
""",file=f)

In [44]:
subprocess.run("vmd -dispdev text -e solute.tcl",shell=True)

Info) VMD for LINUXAMD64, version 1.9.4a57 (April 27, 2022)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 56 CPUs.
Info)   CPU features: SSE2 SSE4.1 AVX AVX2 FMA F16 HT 
Info) Free system memory: 56GB (88%)
Info) No CUDA accelerator devices available.
Info) Dynamically loaded 3 plugins in directory:
Info) /usr/local/lib/vmd/plugins/LINUXAMD64/molfile
ERROR) No molecules loaded.mol operates on one molecule only.
ERROR) No molecules loaded.
slab.pdb
Info) Using plugin pdb for structure file slab.pdb
Info) Using plugin pdb for coordinates from file slab.pdb
Info) Finished with coordinate file slab.pdb.

{0 270} {1 271} {2 272} {3 273} {4 274} {5 275} {6 276} {7 277} {8 278} {9 279} {10 280} {11 281} {12 282} {13 283} {14 284} {15 285} {16 286} {17 287} {18 288} {19 289} {20 290} {21 291} {22 292} {23 293} {24 294} {25 295} {26 296} {27 297} {28 298} {29 299} {30 300} {31 301} {32 302} {33 303} {34 304} {35 305} {36 306} {37 307} {38 308} {39 309} {40 310} {41 311} {42 312} {43 313} {44 314} {45 315} {46 316} {47 317} {48 318} {49 319} {50 320} {51 321} {52 322} {53 323} {486 55} {486 56} {487 54} {487 57} {488 59} {488 60} {489 58} {489 61} {490 63} {490 64} {491 62} {491 65} {492 67} {492 68} {493 66} {493 69} {494 71} {494 72} {495 70} {495 73} {496 75} {496 76} {497 74} {497 77} {498 79} {498 80} {499 78} {499 81} {500 83} {500 84} {501 82} {501 85} {502 87} {502 88} {503 86} {503 89} {504 91} {504 92} {505 90} {505 93} {506 95} {506 96} {507 94} {507 97} {508 99} {508 100} {509 98} {509 101} {510 103} {510 104} {511 102} {511 105} {512 107} {512 108} {513 106} {513 109} {514 111} 

CompletedProcess(args='vmd -dispdev text -e solute.tcl', returncode=0)

# write the force fields file

In [45]:
with open("in.FAPI","w") as f:
    print("""
bond_style          harmonic
angle_style         harmonic
dihedral_style      opls 
improper_style      cvff
pair_style          hybrid lj/cut/coul/long 10.0 8.0 buck/coul/long 10.0 8.0
kspace_style        pppm 1e-4
special_bonds       amber
variable x          equal 0.966 
variable xA         equal 0.9666
variable APb        equal "70359906.629702/v_xA"
variable rPb        equal "0.131258*v_x"
variable rhoPb      equal "0.0*v_x"
variable xR         equal  0.9666 
variable xAR        equal  0.6
variable APbI       equal "103496.133010/v_xAR"
variable rPbI       equal "0.321737*v_xR"
variable rhoPbI     equal "0.0*v_xR"
variable xIR        equal 0.92 
variable xAIR       equal 1.2
variable AI         equal "22793.338582/v_xAIR"
variable rI         equal "0.482217*v_xIR"
variable rhoI       equal "696.94952*v_xIR"

# Force field
pair_coeff 1 1 lj/cut/coul/long     0.076           3.55                 #C-C
pair_coeff 2 2 lj/cut/coul/long     0.0157          1.06910              #H-H
pair_coeff 3 3 lj/cut/coul/long     0.030           2.50                 #Ha-Ha
pair_coeff 4 4 buck/coul/long       ${AI}           ${rI}      ${rhoI}   #I-I
pair_coeff 5 5 lj/cut/coul/long     0.1700          3.25000              #N-N
pair_coeff 6 6 buck/coul/long       ${APb}          ${rPb}     ${rhoPb}  #Pb-Pb
pair_coeff 2 4 lj/cut/coul/long     0.0574          2.75000              #I-H
pair_coeff 1 4 buck/coul/long       112936.714213   0.342426   0.000000  #I-C
pair_coeff 3 4 lj/cut/coul/long     0.0574          3.10000              #I-Ha
pair_coeff 4 5 buck/coul/long       112936.714213   0.342426   0.000000  #I-N
pair_coeff 4 6 buck/coul/long       ${APbI}         ${rPbI}    ${rhoPbI} #Pb-I
pair_coeff 5 6 buck/coul/long       32690390.937995 0.150947   0.000000  #Pb-N
pair_coeff 2 6 lj/cut/coul/long     0.0140          2.26454              #Pb-H
pair_coeff 1 6 buck/coul/long       32690390.937995 0.150947   0.000000  #Pb-C
pair_coeff 3 6 lj/cut/coul/long     0.0140          2.70999              #Pb-Ha

#Bond Coeffs
bond_coeff    1   340.4000   1.0800     #C-Ha
bond_coeff    2   448.0000   1.3650     #C-N
bond_coeff    3   434.0000   1.0100     #H-N

#Angle Coeffs
angle_coeff    1   35.000    120.000    #C-N-H    
angle_coeff    2   35.000    113.000    #H-N-H    
angle_coeff    3   35.000    119.100    #Ha-C-N
angle_coeff    4   70.000    120.000    #N-C-N
 
# Dihedral coefficients
dihedral_coeff   1   0.000      3.900      0.000      0.000    #Ha-C-N-H 
dihedral_coeff   2   0.000      3.900      0.000      0.000    #H-N-C-N

#impropers
improper_coeff   1   2.500      -1       2      #C-N-N-Ha 
improper_coeff   2   2.500      -1       2      #N-C-H-H
""",file=f)

# lammps start file with parameters

In [46]:
with open("start.lmp","w") as f:
    print("""
dimension       3
boundary        p p p
units           real
atom_style      full
variable        seed world 1428
variable        temperature equal 300.0
variable        temperature2 equal 300.0
variable        tempDamp equal 100.0 # approx 0.1 ps
variable        pressure equal 1.00
variable        pressureDamp equal 500.0
variable        freq equal 500
read_data       data.FAPI
include         in.FAPI
thermo          ${freq}
thermo_style    custom step temp pe ke etotal press lx ly lz xy xz yz

# Minimization
min_style cg
#fix 1 all box/relax aniso 0.0 vmax 0.105
minimize        1.0e-4 1.0e-6 1000 10000
#unfix 1
write_data      data.min

# NVT
dump            myDump1 all atom 500 out.0.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature} ${tempDamp} ${seed}
fix             2 all shake 0.0001 20 100000 t 2 3
fix             3 all nve
timestep        2.0
velocity        all create ${temperature} ${seed} dist gaussian
run             10000
unfix           1
unfix           2
unfix           3
write_data      data.NVT
undump          myDump1
reset_timestep  0
# NPT
dump            myDump2 all atom 500 out.1.lammpstrj 
fix             1 all temp/csvr ${temperature} ${temperature} ${tempDamp} ${seed}
fix             2 all shake 0.0001 20 100000 t 2 3
fix             3 all nph tri ${pressure} ${pressure} ${pressureDamp} 
fix             4 all momentum 10000 linear 1 1 1
run             50000
unfix           1
unfix           2
unfix           3
unfix           4
undump          myDump2
reset_timestep  0
write_restart   restart.file
write_data      data.eq
""",file=f)

# run lammps

In [ ]:
subprocess.run("lmp_mpi < start.lmp ",shell=True)

LAMMPS (23 Jun 2022)
Reading data file ...
  triclinic box = (-6.495001 -0.653104 -0.901) to (19.486 21.847103 22.806001) with tilt (-12.9905 0 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  648 atoms
  scanning bonds ...
  3 = max bonds/atom
  scanning angles ...
  3 = max angles/atom
  scanning dihedrals ...
  8 = max dihedrals/atom
  scanning impropers ...
  2 = max impropers/atom
  reading bonds ...
  378 bonds
  reading angles ...
  486 angles
  reading dihedrals ...
  432 dihedrals
  reading impropers ...
  162 impropers
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     3 = max # of 1-2 neighbors
     4 = max # of 1-3 neighbors
     6 = max # of 1-4 neighbors
     7 = max # of special neighbors
  special bonds CPU = 0.001 seconds
  read_data CPU = 0.008 seconds
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0.5     
  special bond fa

      5500   311.86047     -31221.616      517.78544     -30703.831      24709.749      25.981001      22.500207      23.707001     -12.9905        0              0            
      6000   280.77217     -31234.512      466.16918     -30768.343      26932.509      25.981001      22.500207      23.707001     -12.9905        0              0            
      6500   291.31906     -31192.479      483.6803      -30708.799      26332.289      25.981001      22.500207      23.707001     -12.9905        0              0            
      7000   312.22024     -31223.237      518.38276     -30704.854      26478.646      25.981001      22.500207      23.707001     -12.9905        0              0            
      7500   311.59648     -31190.291      517.34712     -30672.944      27346.085      25.981001      22.500207      23.707001     -12.9905        0              0            
      8000   291.95261     -31245.614      484.73219     -30760.882      27626.753      25.981001      22.500207   

     10000   294.75714     -31757.968      489.38858     -31268.579      363.15456      30.519326      27.005543      25.627748     -14.921006      0.65793886     4.39511      
     10500   303.94813     -31736.95       504.64849     -31232.302     -924.01077      30.604033      27.032497      25.857911     -15.17082       1.007093       3.9758401    
     11000   302.0925      -31773.235      501.56756     -31271.667     -105.82484      30.612013      27.035835      25.642975     -15.699862      0.75326501     4.5017677    
     11500   289.32323     -31743.153      480.36661     -31262.787      30.822955      30.88843       27.208242      25.684649     -15.380264      0.20165663     3.7656913    
     12000   283.88744     -31769.134      471.34149     -31297.793     -43.182216      31.277237      26.696978      25.983589     -14.969584      0.95870548     2.3354021    
     12500   296.64931     -31708.365      492.53017     -31215.834      484.19387      31.132887      26.729675   

     33500   293.93125     -31761.699      488.01735     -31273.681     -375.68532      30.556807      26.798135      25.706583     -16.263135      0.843397       3.7414377    
     34000   312.28703     -31747.464      518.49366     -31228.97       27.290068      30.657751      26.807893      25.608795     -15.980714      1.4531806      4.8016721    
     34500   294.61577     -31771.148      489.15386     -31281.994     -747.82976      30.67785       26.398618      25.87374      -16.129458      0.14484735     4.2067919    
     35000   292.88078     -31769.347      486.27324     -31283.074     -268.45664      30.246866      27.093896      25.696064     -16.378291      0.84911921     4.8882542    
     35500   301.60186     -31743.756      500.75295     -31243.003      1224.0107      30.195991      26.714923      25.548485     -16.382695      1.0109694      4.4621409    
     36000   311.11421     -31752.477      516.5464      -31235.931     -420.48222      30.215815      27.335728   

# view trajectories

In [3]:
from ase.io import read
traj = read('out.1.lammpstrj', index=":", parallel=True)
view(traj)

<Popen: returncode: None args: ['/home/ahlawat/miniconda3/bin/python', '-m',...>

## Ongoing try to make lammps data file from ase for molecules, solids, etc.

In [ ]:
# unit_cell = FA_replaced_structure
# rep1 = 1
# rep2 = 1
# rep3 = 1
# i = 0
# j = 0
# k = 0
# x = 0 
# y = 0
# supercell = unit_cell.repeat((rep1,rep2,rep3))
# supercell = sort(supercell)
# num = len(supercell.get_chemical_symbols())
# while k < num:
#     cutOff = neighborlist.natural_cutoffs(supercell)
#     nl = neighborlist.NeighborList(cutOff, self_interaction=False, bothways=True)
#     nl.update(supercell)
#     if(supercell.get_chemical_symbols()[k] == 'H'):
#         x = nl.get_neighbors(k)
#         y = x[0]
#         for j in y:
#             if(supercell.get_chemical_symbols()[j] == 'C'):
#                 supercell.symbols[k] = 'He'          
#     k = k + 1
    
# supercell = sort(supercell)
# view(supercell)
        
# # set point charges
# i = 0
# j = 0
# k = 0
# x = 0 
# y = 0
# num_atoms = len(supercell.get_chemical_symbols())
# charge_array = [0]*num_atoms
# while i < num_atoms:
#     if(supercell.get_chemical_symbols()[i] == 'C'):
#         charge_array[i]= 0.5671
#     if(supercell.get_chemical_symbols()[i] == 'N'):
#         charge_array[i]= -0.8245
#     if(supercell.get_chemical_symbols()[i] == 'He'):
#         charge_array[i]= 0.2225
#     if(supercell.get_chemical_symbols()[i] == 'H'):
#         charge_array[i]= 0.4648
#     if(supercell.get_chemical_symbols()[i] == 'I'):
#         charge_array[i]= -1.0
#     if(supercell.get_chemical_symbols()[i] == 'Pb'):
#         charge_array[i]= 2.0
#     i = i + 1

# supercell.set_initial_charges(charges=charge_array)
# #view(supercell)
# print(supercell) 

# write_lammps_data('data.FAPI', supercell, atom_style = 'full', force_skew=True, units='real')

In [ ]:
# def get_angles(bonds):
#     """ Iterate through bonds to get angles.
#     Bonds should contain no duplicates.
#     """
#     angles = []
#     for i1, b1 in enumerate(bonds):
#         for i2, b2 in enumerate(bonds):
#             if i2 > i1:
#                 shared_atom = list(set(b1) & set(b2))
#                 if len(shared_atom) > 0:
#                     atom1 = [b for b in b1 if b != shared_atom[0]][0]
#                     atom2 = [b for b in b2 if b != shared_atom[0]][0]
#                     other_atoms = sorted([atom1, atom2])
#                     angles.append((other_atoms[0], shared_atom[0], other_atoms[1]))
#     return sorted(angles)


# def calculate_angle(p1, p2, p3):
#     """ Calculate angle for three given points in space
#       p2 ->  o
#             / \
#     p1 ->  o   o  <- p3
#     """
#     p1 = np.array(p1)
#     p2 = np.array(p2)
#     p3 = np.array(p3)

#     v21 = p1 - p2
#     v23 = p3 - p2
#     angle = np.arccos(np.dot(v21, v23) / (np.linalg.norm(v21) * np.linalg.norm(v23)))
#     return np.degrees(angle)


# def get_dihedrals(bonds):
#     """ Iterate through bonds to get dihedrals.
#     Bonds should contain no duplicates.
#     """
#     dihedrals = []
#     for i1, middle_bond in enumerate(bonds):
#         atom1, atom2 = middle_bond
#         atom1_bonds, atom2_bonds = [], []
#         for i2, other_bond in enumerate(bonds):
#             if atom1 in other_bond:
#                 atom1_bonds.append(other_bond)
#             elif atom2 in other_bond:
#                 atom2_bonds.append(other_bond)

#         for bond1 in atom1_bonds:
#             for bond2 in atom2_bonds:
#                 atom0 = [b for b in bond1 if b != atom1][0]
#                 atom3 = [b for b in bond2 if b != atom2][0]
#                 dihedral = (min(atom0,atom3), atom1, atom2, max(atom0,atom3))
#                 # Make sure dihedral is not a loop
#                 if len(set(dihedral)) == 4:
#                     dihedrals.append(tuple(dihedral))

#     return sorted(dihedrals)